# Limpieza de Clientes Potenciales en Bogotá
La idea de este script es limpiar los clientes potenciales en bogotá siguiendo estas reglas:

## Limpieza de clientes potenciales por cercania de PIN.
- Se va a evaluar el geohash de 9 digitos de cada uno de los clientes potenciales, esto es el equivalente a un recuadro de 5x5m en un mapa. Si este recuadro hace match con el geohash de algún otro cliente, entonces se borra.

## Limpieza de potenciales por CRM
- No es claro como usar el CRM para limpiar clientes potenciales pues los clientes potenciales no tienen CRM.

## Limpieza de potenciales por base de llamadas
- Se llamaron a los clientes potenciales y se les hizo preguntas. Si la rta es que ya están registrados en al app, se eliminan.

> Se obtiene la base de clientes con el siguiente codigo sql en Snowflake

~~~sql
SELECT 
      id,
      source_client,
      latitude,
      longitude
FROM 
    "ANALYTICS"."PROD_MODELED"."CLIENTS"
WHERE source_country = 'COLOMBIA'
    and warehouse = 'bodega Fontibon'


In [25]:
import pandas as pd
import geohash
import math

clients_base = pd.read_csv('clients_base_fontibon.csv').to_dict(orient = 'index')
drops = []

GeoHash_MotoTul = set()
GeoHash_Clients = set()

for i in clients_base:
    if ( math.isnan(clients_base[i]['lat']) or math.isnan(clients_base[i]['lng'])):
        drops.append(i)
    else:
        try:
            _9GeoHash = geohash.encode(clients_base[i]['lat'], clients_base[i]['lng'], 9)
            clients_base[i]['hash'] = _9GeoHash
            if clients_base[i]['source_client'] == 'Client':
                GeoHash_Clients.add(_9GeoHash)
            else:
                GeoHash_MotoTul.add(_9GeoHash)
        except:
            print(clients_base[i])

d = GeoHash_Clients.intersection(GeoHash_MotoTul)

for i in clients_base:
    if i in drops:
        pass
    else:
        if (clients_base[i]['hash'] in d and clients_base[i]['source_client'] == 'MotoTul') :
            drops.append(i)

mototul_ids = []
for i in drops:
    mototul_ids.append(clients_base[i]['id'])

In [26]:
import json

with open('zone_clean.json', 'w') as fp:
    json.dump(mototul_ids, fp)

In [27]:
calls = [922,
1359,
1391,
1400,
1402,
1424,
1425,
1451,
1458,
1479,
1511,
1528,
1556,
1558,
1609,
1614,
1623,
1625,
1659,
1679,
1682,
1708,
1714,
1731,
1735,
1741,
1744,
1751,
1802,
1819,
1845,
1863,
1864,
1936,
1949,
2028,
2137,
2323,
2356,
2358,
2377,
2393,
2412,
2440,
2445,
2449,
2460,
2501,
2526,
2582,
2611,
2612,
2618,
2638,
2643,
2662,
2668,
2675,
2676,
2679,
2683,
2689,
2707,
2708,
2722,
2779,
2781,
2788,
2827,
2828,
2829,
2831,
2851,
2853,
2857,
2858,
2864,
2865,
2869,
2898,
2908,
2920,
2926,
2958,
2978,
2988,
2995,
3005,
3008,
3023,
3034,
3090,
3097,
3102,
3107,
3110,
3146,
3171,
3178,
3218,
3219,
3276,
3310,
3397,
3415,
3657,
3664,
3679,
3685,
3694,
3701,
3706,
3707,
3708,
3709,
3710,
3719,
3722,
3724,
3726,
3728,
3739,
3740,
3743,
3745,
3747,
3753,
3754,
3756,
3757,
3758,
3759,
3760,
3765,
3817,
3891,
3954,
3959,
3962,
3966,
3970,
3974,
3976,
3979,
3992,
4003,
4015]

with open('calls_clean.json', 'w') as fp:
    json.dump(calls, fp)